In [ ]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [ ]:
!pip install black

In [ ]:
pip install --upgrade tensorflow

## 1. Import dependencies

In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import time

## 2. Keypoints with Mp Holistic

In [2]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [38]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)  #MODEL PREDICTION
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [39]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [40]:
def draw_styled_landmarks(image,results):
    #draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,256,120), thickness=1, circle_radius=1), #connecting dots
                             mp_drawing.DrawingSpec(color=(50,256,200), thickness=1, circle_radius=1)) #connecting lines
    # draw pos connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(10,100,120), thickness=2, circle_radius=4), #connecting dots
                             mp_drawing.DrawingSpec(color=(20,20,80), thickness=2, circle_radius=2)) #connecting lines
    #draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(100,20,200), thickness=2, circle_radius=4), #connecting dots
                             mp_drawing.DrawingSpec(color=(120,10,240), thickness=2, circle_radius=2)) #connecting lines
    #draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(10,20,240), thickness=2, circle_radius=4), #connecting dots
                             mp_drawing.DrawingSpec(color=(50,10,10), thickness=2, circle_radius=2)) #connecting lines

In [6]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame= cap.read()
        
        #make_detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #show to screen
        cv2.imshow("OpenCv Feed", image)
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## 3. Extract Keypoint Values

In [9]:
len(results.pose_landmarks.landmark)

33

In [ ]:
left = []
for res in results.right_hand_landmarks.landmark:
    test=np.array([res.x,res.y, res.z])
    left.append(test)


In [41]:
def extract_keypoints(results):
    #flatten consists all of the array in one big array
    pose=np.array([[res.x,res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)

    lh=np.array([[res.x,res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)

    rh=np.array([[res.x,res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)

    face=np.array([[res.x,res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    
    return np.concatenate([pose, face, lh, rh])

## 4. Setup Folders For Collection

In [5]:
#Demonstrate the exported folder, np array
DATA_PATH=os.path.join('MP_Data')

#Actions that we try to detect
actions = np.array(['Hello', 'Thanks', 'IloveYou', 'Hi5'])

#30 videos worth of data
no_sequences= 30

#each video 30 frames in length
sequence_length = 30

In [6]:
#make 4 folder for 4 different actions in each folder
#hello->
#seq0 (30 frames / keypoints)..... seq 29
#....
#hi5->
#seq0 (....) ... seq 29

In [13]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect Keypoints for training and testing

In [14]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #loop through actions
    for action in actions:
        #loop through each sequence aka videos
        for seq in range(no_sequences):
            #loop through video length aka seq length
            for frame_num in range(sequence_length):
                
                
                ret, frame= cap.read()

                #make_detections
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                #draw landmarks
                draw_styled_landmarks(image, results)
                
                #apply wait logic:
                if frame_num == 0:
                    cv2.putText(image, "Starting Collection", (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
                    cv2.putText(image, 'collecting video for {} video num: {}'.format(action, seq) , (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    #show to screen
                    cv2.imshow('OpenCv Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'collecting video for {} video num: {}'.format(action, seq) , (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    #show to screen
                    cv2.imshow("OpenCv Feed", image)
                    
                #NEw Extract points
                keypoints= extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(seq), str(frame_num))
                np.save(npy_path, keypoints)
                

                #break gracefully
        
                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

## 6. PreProcess Data Create Labels and Features

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [7]:
label_map = {label: num for num, label in enumerate(actions)}

In [8]:
label_map

{'Hello': 0, 'Thanks': 1, 'IloveYou': 2, 'Hi5': 3}

In [9]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window =[]
        for frame_num in range(sequence_length):
            res= np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [10]:
np.array(sequences).shape

(120, 30, 1662)

In [11]:
np.array(labels).shape

(120,)

In [12]:
y= to_categorical(labels).astype(int)

In [14]:
X= np.array(sequences)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## 7. Train and Build LSTM Network

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import LSTM, Dense

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [19]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [20]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200


2022-03-06 11:55:12.155312: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22736160 exceeds 10% of free system memory.


4/4 [==============================] - 5s 172ms/step - loss: 2.6678 - categorical_accuracy: 0.2632
Epoch 2/200
4/4 [==============================] - 1s 164ms/step - loss: 2.3696 - categorical_accuracy: 0.2632
Epoch 3/200
4/4 [==============================] - 1s 159ms/step - loss: 1.3905 - categorical_accuracy: 0.3421
Epoch 4/200
4/4 [==============================] - 1s 162ms/step - loss: 1.3786 - categorical_accuracy: 0.3596
Epoch 5/200
4/4 [==============================] - 1s 151ms/step - loss: 1.3763 - categorical_accuracy: 0.2895
Epoch 6/200
4/4 [==============================] - 1s 217ms/step - loss: 1.3779 - categorical_accuracy: 0.3246
Epoch 7/200
4/4 [==============================] - 1s 174ms/step - loss: 1.3407 - categorical_accuracy: 0.2368
Epoch 8/200
4/4 [==============================] - 1s 166ms/step - loss: 1.3477 - categorical_accuracy: 0.3246
Epoch 9/200
4/4 [==============================] - 1s 170ms/step - loss: 1.3375 - categorical_accuracy: 0.3596
Epoch 10/200


4/4 [==============================] - 1s 298ms/step - loss: 0.4184 - categorical_accuracy: 0.8421
Epoch 75/200
4/4 [==============================] - 1s 330ms/step - loss: 0.3602 - categorical_accuracy: 0.9211
Epoch 76/200
4/4 [==============================] - 2s 444ms/step - loss: 0.3276 - categorical_accuracy: 0.9298
Epoch 77/200
4/4 [==============================] - 2s 387ms/step - loss: 0.3049 - categorical_accuracy: 0.9298
Epoch 78/200
4/4 [==============================] - 1s 324ms/step - loss: 0.2843 - categorical_accuracy: 0.9035
Epoch 79/200
4/4 [==============================] - 1s 277ms/step - loss: 0.4653 - categorical_accuracy: 0.7982
Epoch 80/200
4/4 [==============================] - 1s 236ms/step - loss: 0.3240 - categorical_accuracy: 0.8947
Epoch 81/200
4/4 [==============================] - 1s 211ms/step - loss: 0.2552 - categorical_accuracy: 0.9298
Epoch 82/200
4/4 [==============================] - 1s 239ms/step - loss: 0.2566 - categorical_accuracy: 0.9386
Epoch

4/4 [==============================] - 1s 273ms/step - loss: 0.1312 - categorical_accuracy: 0.9561
Epoch 147/200
4/4 [==============================] - 1s 270ms/step - loss: 0.1512 - categorical_accuracy: 0.9474
Epoch 148/200
4/4 [==============================] - 1s 205ms/step - loss: 0.2945 - categorical_accuracy: 0.8772
Epoch 149/200
4/4 [==============================] - 1s 297ms/step - loss: 0.6996 - categorical_accuracy: 0.7982
Epoch 150/200
4/4 [==============================] - 1s 203ms/step - loss: 0.5082 - categorical_accuracy: 0.7807
Epoch 151/200
4/4 [==============================] - 1s 213ms/step - loss: 0.3115 - categorical_accuracy: 0.9035
Epoch 152/200
4/4 [==============================] - 1s 182ms/step - loss: 0.2319 - categorical_accuracy: 0.9649
Epoch 153/200
4/4 [==============================] - 1s 177ms/step - loss: 0.2685 - categorical_accuracy: 0.9211
Epoch 154/200
4/4 [==============================] - 1s 196ms/step - loss: 0.2770 - categorical_accuracy: 0.95

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 596,708
Trainable params: 596,708
Non-tr

## 8. Make Predictions

In [25]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[2])]

'Hello'

In [31]:
actions[np.argmax(y_test[2])]

'Hello'

9. Save Weights

In [32]:
model.save('action.h5')

In [ ]:
# to load the model ==== model.load_weights('action.h5')

# 10. Evaluation using Confusing Matrix and Accuracy

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [34]:
y_pred = model.predict(X_test)

In [35]:
ytrue= np.argmax(y_test, axis=1).tolist()
y_pred = np.argmax(y_pred, axis=1).tolist()

In [36]:
multilabel_confusion_matrix(ytrue, y_pred)

array([[[4, 0],
        [0, 2]],

       [[5, 0],
        [0, 1]],

       [[3, 0],
        [0, 3]]])

In [48]:
accuracy_score(ytrue, y_pred)

1.0

## 11. Test in Real TIme

In [99]:
colors = [(245,117,16), (117,245,16), (16,117,245), (16,245,16)]
def probability_visualization(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [98]:
#Detection Variable
threshold=0.6
sequence = []
sentence = []
prediction =[]

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame= cap.read()
        
        #make_detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #prediction Logic
        keypoints= extract_keypoints(results)
        sequence.append(keypoints)
        sequence= sequence[-30:]
        
        if len(sequence)==30:
            res= model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            prediction.append(np.argmax(res))
            
            #Visualization Logic
            if np.unique(prediction[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)].any() > threshold:
                    if (len(sentence)) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else: 
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence=sentence[-5:]
            
            # Viz probabilities
            image = probability_visualization(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640,40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                
        
        #show to screen
        cv2.imshow("OpenCv Feed", image)
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
IloveYou
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Thanks
Thanks
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5
Hi5


In [95]:
cap.release()
cv2.destroyAllWindows()